# <b>Object Detection with Vertex AI AutoML</b>

## <b>Learning Objectives</b> ##

1. Learn how to create a managed dataset on Vertex AI for object detection
1. Learn how to train an AutoML object detection model on Vertex
1. Learn how to evaluate a model trained with AutoML on Vertex
1. Learn how to deploy a model trained with AutoML to a Vertex Endpoint
1. Learn how to predict on new data with deployed model


In this notebook we will use Vertex AI to train an AutoML model capable of detecting multiple objects in a given image and provide information about the objects and their location within the image.

We will start by creating a Managed Dataset on Vertex AI and importing a publicly available set of images into it. After that we will train, evaluate and deploy the AutoML model to a Vertex Endpoint. Finally we will send prediction requests to our model.

### Setup

In [1]:
import os
from datetime import datetime

from google.cloud import aiplatform

PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
BUCKET = PROJECT  # defaults to PROJECT
REGION = "us-central1"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

aiplatform.init(project=PROJECT, location=REGION)

## Step 1: Prepare and Format Training Data

The first step in creating a Managed Datset on Vertex AI is to prepare the training data. In this case the training dataset is composed of images along with information identifying the location (through bounding boxes coordinates) and type of objects (through labels) in the images. 
Here are some constraints some general rules for preparing an Managed Datset for object detection:

* The following image formats are supported: JPEG, PNG, GIF, BMP, or ICO. Maximum file size is 30MB per image.

* It is recommended to have about 1000 training images per label (i.e. object type you want to detect in the images). For each label you must have at least 10 images, each with at least one annotation (bounding box and the label). In general, the more images per label you have the better your model will perform.

### Create a CSV file with image URIs and labels

Once your image files have been uploaded to a Cloud Storage bucket, you must create a CSV (or JSONL) file that lists the image URIs, along with bounding box information and the object labels. The CSV file will contain one row per bounding box in the image, so an image that has two bounding boxes will have two corresponding rows in the CSV file sharing the same image URI. 

In the example below, rows 1 and 2 reference the same image that has 2 annotations 
`(car,0.1,0.1,,,0.3,0.3,,)` and  `(bike,.7,.6,,,.8,.9,,)`. The first element of the annotation
is the object label in the bounding box, while the rest are the coordinates of the bounding box
within the image (see below for details).


Row 3 refers to an image that has only 1 annotation `(car,0.1,0.1,0.2,0.1,0.2,0.3,0.1,0.3)`, while row 4 references an image with no annotations.

The first column corresponds to the data split (this is optional - if not provided autoML will do a 80/10/10 split), the second column to the image URI, and the last columns hold the annotations. 

Bounding boxes can be in one of two formats:
- Containing 2 vertices (x,y coordinates of diagonally opposite points of the rectangle): `(x_min, y_min, , ,x_max,y_max,,)` OR
- Containing all 4 vertices`(x_min,y_min,x_max,y_min,x_max,y_max,x_min,y_max)`
**Example:**

```bash
TRAIN,gs://folder/image1.png,car,0.1,0.1,,,0.3,0.3,,
TRAIN,gs://folder/image1.png,bike,.7,.6,,,.8,.9,,
UNASSIGNED,gs://folder/im2.png,car,0.1,0.1,0.2,0.1,0.2,0.3,0.1,0.3
TEST,gs://folder/im3.png,,,,,,,,,
```

Each row above has these columns:

1. <b>Which dataset is the content in the row being assigned to.</b> - `TRAIN`, `VALIDATE`, `TEST` or `UNASSIGNED`
1. <b>What content is being annotated.</b> - It contains the GCS URI for the image
1. <b>A label that identifies how the object is categorized.
1. <b>A bounding box for an object in the image.</b>

For this lab, we will use a CSV that has already been created for us. The image URIs in the CSV point to publically avaliable images in GCS and contain annotations related to the foods/ingredients in each image.

Make sure the GCS bucket exists (if not then create it), then copy the CSV to your GCS bucket.

In [ ]:
%%bash
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
   echo -e "Bucket gs://${BUCKET} already exists."
    
else
   echo "Creating a new GCS bucket."
   gsutil mb -l ${REGION} gs://${BUCKET}
   echo -e "\nHere are your current buckets:"
   gsutil ls
fi

Take a look at the first few rows of the CSV file, then copy it to your GCS bucket

In [ ]:
!head ../dataset_import_files/salads.csv

In [ ]:
!gsutil cp ../dataset_import_files/salads.csv gs://{BUCKET}/

## Step 2: Create the Managed Dataset

Next step is to create a Managed Dataset on Vertex AI for object detection.
<br/>
**NOTE**: This can take between 5-10 minutes.

In [ ]:
DATASET_NAME = "salad_dataset"

ds = aiplatform.ImageDataset.create(
    display_name=DATASET_NAME,
    gcs_source=f"gs://{BUCKET}/salads.csv",
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.bounding_box,
)

## Step 3: Train AutoML Model
Launch the training job. We will set the following parameters in `job.run()`
- `dataset`: The managed dataset we just created
- `model_display_name`: The display name of the model. The trained model will become a Vertex Model object after training is finished.
- `training_fraction_split`: Percentage of data to train on. Explicitly set to 0.8 (80%).
- `validation_fraction_split`: Percentage of data to validate on. Explicitly set to 0.1 (10%).
- `test_fraction_split`: Percentage of data to test on. Explicitly set to 0.1 (10%).
- `budget_milli_node_hours`: Milli-node hours for training job (e.g. 1000 means 1 node hour). Minimum for AutoML Object Detection training jobs is 20000 (20 node hours). 
- `disable_early_stopping`: Boolean. Explicitly set to False (i.e. Enable early stopping).  

In [ ]:
MODEL_DISPLAY_NAME = "salad"

job = aiplatform.AutoMLImageTrainingJob(
    display_name="salad_" + TIMESTAMP, prediction_type="object_detection"
)

model = job.run(
    # TODO: Launch the training job
    # Set the train split to 80% with 10% for both validation and testing
    # Set the budget to 20 node hours
    # Enable early stopping
    model_display_name=MODEL_DISPLAY_NAME,
)

**NOTE:** Model training can take between 1.5-2.5 hours to complete. You must wait for model to finish training before moving forward. 

Retrieve the model resource and get the evaluation metrics. We will look at the [Mean Average Precision](https://blog.paperspace.com/mean-average-precision/). [Here](https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/) is a good resource to learn more about evaluation metrics for object detection models. 

In [ ]:
# Get model resources by display name
models = aiplatform.Model.list(filter="display_name=salad")
salad_model = models[0]

# Get model evaluation
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=salad_model.resource_name
)
evals = list(model_evaluations)[0]

for metric in list(evals.metrics["boundingBoxMetrics"]):
    try:
        print(f"Mean Average Precision (mAP): {metric['meanAveragePrecision']}")
    except KeyError:
        print("IOU threshold is too high to detect positive cases.")

    print(f"Intersection over union (IoU): {metric['iouThreshold']}\n")

## Step 6: Deploy the model

Once we are happy with the performance of our trained model, we can deploy it so that it will be
available for predictions through an API. 

In [ ]:
ENDPOINT_DISPLAY_NAME='salad_model_endpoint'

# TODO: Deploy the model to an endpoint 
endpoint =

## Step 7: Send prediction request

In this example we will invoke an individual prediction from an image that is stored in our project's Cloud storage bucket.
Object detection models output many bounding boxes for an input image. For the output we are expecting that each box comes with:
1. a label and 
1. a score of confidence.

Preparing the data to get a prediction entails the following:
1. Encode the image with `base64.b64encode` which returns the encoded bytes. Then apply `.decode("utf-8")` to return the decoded string.
1. Create the `ImageObjectDetectionPredictionInstance`
1. Create the `ImageObjectDetectionPredictionParams`. This is namely used to set a confidence threshold.
1. Make the prediction request


In [ ]:
import base64

from google.cloud.aiplatform.gapic.schema import predict

TEST_IMAGE = "../test_images/salad.jpg"  # Replace with a Cloud storage bucket uploaded image of your choice

# Read the file.
with open(TEST_IMAGE, "rb") as f:
    file_content = f.read()

encoded_content = base64.b64encode(file_content).decode("utf-8")
instance = predict.instance.ImageObjectDetectionPredictionInstance(
    content=encoded_content
).to_value()

parameters = predict.params.ImageObjectDetectionPredictionParams(
    confidence_threshold=0.7
).to_value()

response = endpoint.predict(instances=[instance], parameters=parameters)
preds = response.predictions[0]
preds

Now that we have the response object from the deployed model, we can inspect its predictions (i.e., the
bounding boxes and objects that the model has detected from the images we sent to it in the cell above):

`preds` is a Python dictionary with keys:
* `bboxes`: A list of predicted bounding box coordinates
* `displayNames`: A list of predicted labels for the bounding boxes
* `confidences`: A list of the prediction confidences.

In [ ]:
from itertools import cycle

import matplotlib.patches as patches
import matplotlib.pyplot as plt
from PIL import Image

# To generate a different colors for each label
cycol = cycle("bgrcmk")

img = Image.open(TEST_IMAGE)
fig, ax = plt.subplots()
ax.imshow(img)

for i in range(len(preds["displayNames"])):
    bbox = preds["bboxes"][i]
    display_name = preds["displayNames"][i]
    confidence = preds["confidences"][i]
    xy = (int(bbox[0] * img.width), int(bbox[2] * img.height))
    w = int(img.width * (bbox[1] - bbox[0]))
    h = int(img.height * (bbox[3] - bbox[2]))
    r = patches.Rectangle(
        xy,
        w,
        h,
        linewidth=1,
        facecolor="none",
        label=f"{display_name}: {confidence:.3f}",
        edgecolor=next(cycol),
    )
    ax.add_patch(r)

plt.legend()
plt.show()

Congrats! In this lab, you created a Vertex AI Managed Dataset for object detection. You then trained an AutoML model and deployed that trained model to an endpoint to serve predictions with.